# The Cost of Copy

![performance-move](img/performance-move.png)

<div style="text-align:center">
    <a style="text-align:center" href="http://quick-bench.com/0wIVJCnNL8z7oRGL7SpR24d2ytQ">Cost of Copy</a>
</div>


- To find the places where copy-ctor and copy-assignment are called, mark them deprecated
    - Along with setting breakpoints

In [1]:
class example {
public:
    example() = default;
    [[deprecated]] example(const example&) = default;
    example(example&&) noexcept = default;
    [[deprecated]] example& operator=(const example&) = default;
    example& operator=(example&&) noexcept = default;
};

In [2]:
example x;
example y = x;

input_line_8:3:13: warning: 'example' is deprecated [-Wdeprecated-declarations]
example y = x;
            ^
input_line_7:4:7: note: 'example' has been explicitly marked deprecated here
    [[deprecated]] example(const example&) = default;
      ^


## Copy Location and Pass By Value

- When passing an lvalue as a by value argument, the copy occurs at the call site
- For non-inline functions this may cause binary bloat
    - Can use pass-by rvalue and lvalue reference for such cases

In [3]:
class sink {
    example _m;
public:
    void set(example e) { _m = std::move(e); }
};

In [4]:
sink s;
s.set(x);

input_line_10:3:7: warning: 'example' is deprecated [-Wdeprecated-declarations]
s.set(x);
      ^
input_line_7:4:7: note: 'example' has been explicitly marked deprecated here
    [[deprecated]] example(const example&) = default;
      ^


In [5]:
.undo 2

In [6]:
class sink {
    example _m;
public:
    void set(const example& e) { _m = e; }
    void set(example&& e) { _m = std::move(e); }
};

input_line_11:4:37: warning: 'operator=' is deprecated [-Wdeprecated-declarations]
    void set(const example& e) { _m = e; }
                                    ^
input_line_7:6:7: note: 'operator=' has been explicitly marked deprecated here
    [[deprecated]] example& operator=(const example&) = default;
      ^


In [7]:
sink s;
s.set(x);